# Imports

In [1]:
import findspark

findspark.init('C:/spark')

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Create a session

In [5]:
spark = SparkSession.builder.appName('als').getOrCreate()

# Read data

In [6]:
data = spark.read.csv('../../data/movielens_ratings.csv',
                      inferSchema=True,
                      header=True)

In [8]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [10]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



# Modeling

In [11]:
training, test = data.randomSplit([0.8, 0.2])

In [14]:
als = ALS(maxIter=5, regParam=0.01,
          userCol='userId',
          itemCol='movieId',
          ratingCol='rating')

In [15]:
model = als.fit(training)

In [16]:
predictions = model.transform(test)

In [17]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      6|   1.0|    28|-0.31618947|
|      1|   1.0|    26|   2.057168|
|      4|   1.0|    12|  1.8709027|
|      6|   1.0|    12|  1.3979996|
|      0|   1.0|    22|  1.7654188|
|      6|   2.0|    22|  2.5012531|
|      2|   2.0|     1|  3.4161732|
|      1|   1.0|    20|-0.45849815|
|      2|   2.0|    20|    5.06528|
|      0|   1.0|    19|  1.1743954|
|      3|   1.0|    17|0.048907995|
|      4|   1.0|     9| 0.17049532|
|      1|   1.0|     4|  0.5272999|
|      2|   1.0|    23|  1.6311911|
|      2|   2.0|     7|  3.3726223|
|      7|   1.0|     7| 0.70914155|
|      2|   1.0|    25|  2.0681825|
|      7|   1.0|    24|   2.848649|
|      4|   1.0|    29|  3.3607392|
|      0|   1.0|    11|  0.9651487|
+-------+------+------+-----------+
only showing top 20 rows



In [18]:
evaluator = RegressionEvaluator(metricName='rmse',
                                labelCol='rating',
                                predictionCol='prediction')

In [19]:
rmse = evaluator.evaluate(predictions)

In [20]:
print('RMSE')
print(rmse)

RMSE
1.6978676979303058


In [22]:
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])
single_user

DataFrame[movieId: int, userId: int]

In [23]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     11|    11|
|     21|    11|
|     27|    11|
|     30|    11|
|     36|    11|
|     38|    11|
|     39|    11|
|     41|    11|
|     61|    11|
|     62|    11|
|     66|    11|
|     67|    11|
|     71|    11|
|     75|    11|
|     86|    11|
|     97|    11|
+-------+------+



In [24]:
recommendations = model.transform(single_user)

In [25]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     66|    11|  4.3951917|
|     36|    11|  3.2378764|
|     27|    11|  3.1401439|
|     39|    11|  3.0817382|
|     75|    11|  2.9182405|
|     21|    11|  2.7671561|
|     67|    11|   2.358541|
|     11|    11|  2.0719857|
|     86|    11|  1.7532898|
|     61|    11|  1.4616084|
|     30|    11|  1.2007986|
|     41|    11|  1.0759029|
|     71|    11|  1.0587294|
|      0|    11|  0.9651487|
|     97|    11| 0.06765646|
|     38|    11| -0.3069635|
|     62|    11|-0.73845834|
+-------+------+-----------+

